# HA 3

Ozmaden Deniz
194

### Problem 1 (1 point)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1) Implement function to generate common secret key within multiplicative group of given Finite field with known generator. 

*Note. You can assume that all the numbers are small, for example, less than 1000.*

In [210]:
def generate_common_secret_key(p, generator, alices_private_key, bobs_open_key=None, bobs_private_key=None):
    if (bobs_open_key != None):
        return (bobs_open_key ** alices_private_key) % p
    elif (bobs_private_key != None):
        generated_bobs_open_key = (generator ** bobs_private_key) % p
        return (generated_bobs_open_key ** alices_private_key) % p
    else: 
        raise Exception('No private key or public key provided for Bob!')

2) Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [211]:
generate_common_secret_key(p = 17, generator = 11, alices_private_key = 7, bobs_open_key = 11)

3

### Problem 2 (3 points)

El Gamal protocol is widely used in cryptography. In this task we will ask you to implement your own El-Gamal encryption scheme on Python.

1) Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

*Note. You can assume that all the numbers are small, for example, less than $2^{32}$. But you **must** use a primality test as a part of the function to get a full score.*

In [212]:
# optimized search using fact that all primes greater than 3 are of the form 6k ± 1
def is_prime(n: int) -> bool:
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0 or n % 3 == 0:
        return False
    limit = int(n**0.5)
    for i in range(5, limit+1, 6):
        if n % i == 0 or n % (i+2) == 0:
            return False
    return True

In [213]:
import random

def miller_rabin_test(n, k=40):
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0 or n % 3 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

In [214]:
def generate_prime(limit):
    num = -1
    while (not is_prime(num) or not miller_rabin_test(num)):
        num = random.randrange(1, limit)
    return num

In [215]:
generate_prime(2**32)

1396281641

2) Implement functions that realize the encryption and decryption in El Gamal protocol. (1 points)

In [216]:
def generate_keys(p, generator):
    private_key = random.randrange(0, generator)
    public_key = (generator ** private_key) % p
    return private_key, public_key

In [217]:
# https://www.educative.io/answers/what-is-the-el-gamal-encryption

p = generate_prime(2**10)
generator = random.randrange(1, p)

# Alice generates own private key and public key
alices_private_key, alices_public_key = generate_keys(p, generator)

# Bob generates own private key and public key
bobs_private_key, bobs_public_key = generate_keys(p, generator)

shared_secret_key = generator ** (bobs_private_key * alices_private_key) % p

In [218]:
def elgamal_encypt(message, shared_secret_key, p):
    encrypted_message = list()
    for e in message:
        encrypted_message.append((e * shared_secret_key) % p)
    return encrypted_message

def elgamal_decrypt(encrypted_message, sender_public_key, receiver_private_key, p):
    decrypted_message = list()
    for e in encrypted_message:
        decrypted_message.append(e * pow(sender_public_key, p-1-receiver_private_key, p) % p)
    return decrypted_message

In [219]:
message = [random.randrange(1, 2**5)]
print(f"Original message is: {message}")

encrypted_message = elgamal_encypt(message, shared_secret_key, p)
print(f"Encrypted message is: {encrypted_message}")

Original message is: [14]
Encrypted message is: [57]


In [220]:
decrypted_message = elgamal_decrypt(encrypted_message, bobs_public_key, alices_private_key, p)
print(f"Decrypted message is: {decrypted_message}")

Decrypted message is: [14]


In [221]:
assert(message == decrypted_message)

3) Calculate Hash of your name by SHA-1 and test El Gamal encryption/decryption functions on its 16-bit prefix. (1 points)

In [222]:
import hashlib

my_name = b"Deniz"
name_hash = hashlib.sha1(my_name).hexdigest() 
print(f"Name hash: {name_hash}")

prefix = int(name_hash, 16) 
print(f"16-bit prefix = {prefix}")

message_array = []
for c in str(prefix):
    message_array.append(int(c))
print(f"Original message array: {message_array}")

encrypted_name = elgamal_encypt(message_array, shared_secret_key, p)
print(f"Encrypted name = {encrypted_name}")

decrypted_name = elgamal_decrypt(encrypted_name, bobs_public_key, alices_private_key, p)
print(f"Decrypted name = {decrypted_name}")

arr_string = ""
for e in decrypted_name:
    arr_string = arr_string + (str(e))

decrypted_name_hash = hex(int(arr_string))
print(f"Decrypted name hash = {decrypted_name_hash[2:]}")

Name hash: 90c1933bb1ae32755dd59ea4f971280e54a78756
16-bit prefix = 826411540640818757174046232635753415468640601942
Original message array: [8, 2, 6, 4, 1, 1, 5, 4, 0, 6, 4, 0, 8, 1, 8, 7, 5, 7, 1, 7, 4, 0, 4, 6, 2, 3, 2, 6, 3, 5, 7, 5, 3, 4, 1, 5, 4, 6, 8, 6, 4, 0, 6, 0, 1, 9, 4, 2]
Encrypted name = [250, 443, 568, 125, 602, 602, 727, 125, 0, 568, 125, 0, 250, 602, 250, 409, 727, 409, 602, 409, 125, 0, 125, 568, 443, 284, 443, 568, 284, 727, 409, 727, 284, 125, 602, 727, 125, 568, 250, 568, 125, 0, 568, 0, 602, 91, 125, 443]
Decrypted name = [8, 2, 6, 4, 1, 1, 5, 4, 0, 6, 4, 0, 8, 1, 8, 7, 5, 7, 1, 7, 4, 0, 4, 6, 2, 3, 2, 6, 3, 5, 7, 5, 3, 4, 1, 5, 4, 6, 8, 6, 4, 0, 6, 0, 1, 9, 4, 2]
Decrypted name hash = 90c1933bb1ae32755dd59ea4f971280e54a78756


### Problem 3 (4 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write Python function that returns all group elements of a certain elliptic curve over a finite field 

1) Write a python function that list all points of elliptic curve $y^2=x^3+7$ over $F_{127}$ (1 point)

*Note. $127 = 2^7-1$ is the fourth Mersenne prime.*

In [223]:
def elliptic_curve_group_elements(p, num):
    ans = []
    for x in range(1, p):
        for y in range(1, p):
            if ((x**3 + num) % p == (y**2) % p):
                ans.append((x, y))
    return ans

In [224]:
q = 127
points = elliptic_curve_group_elements(q, 7)
display(points)

[(1, 32),
 (1, 95),
 (2, 53),
 (2, 74),
 (3, 62),
 (3, 65),
 (4, 43),
 (4, 84),
 (8, 30),
 (8, 97),
 (11, 24),
 (11, 103),
 (12, 46),
 (12, 81),
 (14, 46),
 (14, 81),
 (17, 27),
 (17, 100),
 (18, 39),
 (18, 88),
 (19, 32),
 (19, 95),
 (21, 39),
 (21, 88),
 (24, 49),
 (24, 78),
 (25, 30),
 (25, 97),
 (28, 49),
 (28, 78),
 (32, 3),
 (32, 124),
 (34, 24),
 (34, 103),
 (38, 53),
 (38, 74),
 (39, 12),
 (39, 115),
 (41, 27),
 (41, 100),
 (45, 33),
 (45, 94),
 (46, 51),
 (46, 76),
 (47, 43),
 (47, 84),
 (51, 18),
 (51, 109),
 (52, 36),
 (52, 91),
 (57, 62),
 (57, 65),
 (58, 38),
 (58, 89),
 (60, 19),
 (60, 108),
 (67, 62),
 (67, 65),
 (69, 27),
 (69, 100),
 (70, 19),
 (70, 108),
 (71, 63),
 (71, 64),
 (72, 16),
 (72, 111),
 (75, 49),
 (75, 78),
 (76, 43),
 (76, 84),
 (78, 50),
 (78, 77),
 (79, 63),
 (79, 64),
 (80, 18),
 (80, 109),
 (82, 24),
 (82, 103),
 (84, 16),
 (84, 111),
 (85, 50),
 (85, 77),
 (86, 38),
 (86, 89),
 (87, 53),
 (87, 74),
 (88, 39),
 (88, 88),
 (91, 50),
 (91, 77),
 (93, 3

2) Compare the number of points with Hasse’s estimate $|N-(q+1)|\leq 2{\sqrt  {q}}$. (1 point)

In [225]:
from math import sqrt

number_of_points = len(points)

left = abs(number_of_points - (q + 1))
right = 2 * sqrt(q)

print(left <= right)

True


3) Prove that the point
$A = (19, 32)$ belongs to the elliptic curve and construct a sequence of $B_n = nA, n = 1, ..., 100$. (2 points)

In [226]:
def point_is_on_curve(point, q, num):
    x, y = point
    return (x**3 + num) % q == (y**2) % q

def gcdExtended(a, b):
     if a == 0:
          return b, 0, 1
     gcd, x1, y1 = gcdExtended(b % a, a)
     x = y1 - (b // a) * x1
     y = x1
     return gcd, x, y

A = (19, 32)

print(f"Point (19, 32) is on curve? {point_is_on_curve(A, 127, 7)}")


Point (19, 32) is on curve? True


In [227]:
def double_point(point, q):
     x, y = point
     s = ((3*(x**2)) * (gcdExtended(2*y, q)[1])) % q
     newx = (s**2 - x - x) % q
     newy = (s * (x - newx) - y) % q
     return (newx, newy)

def add_points(first_point, second_point, q):
    x1, y1 = first_point
    x2, y2 = second_point
    s = ((y2 - y1) * ((gcdExtended(x2-x1, q))[1] % q)) % q
    newx = (s**2 - x1 - x2) % q
    newy = (s * (x1 - newx) - y1) % q
    return (newx, newy)

In [228]:
ans = [A]

for i in range(1, 101):
    if (ans[-1] == A):
        new_point = double_point(A, q)
        ans.append(new_point)
    else:
        new_point = add_points(ans[-1], A, q)
        ans.append(new_point)

ans

[(19, 32),
 (11, 24),
 (98, 16),
 (123, 77),
 (27, 28),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 56),
 (69, 103),
 (113, 5

### Problem 4 (2 points)

Let $p = 601$, $q = 7$, $e = 1463$ be the setup of the RSA algorithm. Compute $d$, sign the plane message $m = 58$ and check the signature.

In [229]:
p = 601
q = 7
e = 1463
m = 58

phi = (p-1)*(q-1)
d = pow(e, -1, phi)
print(f"Calculated d = {d}")

print(f"Original message: {m}")

rsa_encrypted_message = m ** e % (p*q)
print(f"Encrypted message: {rsa_encrypted_message}")

rsa_decrypted_message = rsa_encrypted_message ** d % (p*q)
print(f"Decrypted message: {rsa_decrypted_message}")

Calculated d = 1127
Original message: 58
Encrypted message: 2937
Decrypted message: 58


### Problem 5 (4* points)

*bonus problem

Consider an elliptic curve $y^2 = x^3 - 2 x - 1$.

1) (1 point) Plot the curve for $(x, y) \in (-19, 19)$. Plot $$\{(x \mod 19, y \mod 19) \; | \;  (x, y) \in (-19 k, 19 k)^2 \; and \; y^2 = x^3 - 2 x - 1\}$$ for $k = 1, \dots, 5$. 

2) (1 point) Scatter all the integer points of curve $y^2 = x^3 - 2 x - 1$ $\mod 19$. How many points are not in $$\{(x \mod 19, y \mod 19) \; | \;  (x, y) \in (-19 \cdot 5, 19  \cdot 5)^2 \; and \; y^2 = x^3 - 2 x - 1\}?$$ 

3) (2 points) Choose integer examples for all three elliptic curve $y^2 = x^3 - 2 x - 1$ addition cases and illustrate them with (straight) lines.